In [22]:
import torch
import pandas as pd
from pathlib import Path
from transformers import pipeline
import torch.nn as nn
import numpy as np

In [16]:
def run(args):
    print(args)
    # Seeds for reproducibility (By default we use the number 21)

    # Load sales data
    data_folder = "./dataset/"
    train_df = pd.read_csv(Path(data_folder + 'train.csv'), parse_dates=['release_date'])
    test_df = pd.read_csv(Path(data_folder + 'test.csv'), parse_dates=['release_date'])

    # Load category and color encodings
    cat_dict = torch.load(Path(data_folder + 'category_labels.pt'))
    col_dict = torch.load(Path(data_folder + 'color_labels.pt'))
    fab_dict = torch.load(Path(data_folder + 'fabric_labels.pt'))

    print(cat_dict)
    print(col_dict)
    print(fab_dict)

    return cat_dict, col_dict, fab_dict

In [17]:
cat_dict, col_dict, fab_dict = run(5)

5
{'capris': 0, 'cloak': 1, 'culottes': 2, 'doll dress': 3, 'drop sleeve': 4, 'gitana skirt': 5, 'kimono dress': 6, 'long cardigan': 7, 'long coat': 8, 'long dress': 9, 'long duster': 10, 'long skirt': 11, 'long sleeve': 12, 'longuette skirt': 13, 'maxi': 14, 'medium cardigan': 15, 'medium coat': 16, 'medium duster': 17, 'midi skirt': 18, 'miniskirt': 19, 'patterned': 20, 'printed': 21, 'sheath dress': 22, 'shirt dress': 23, 'short cardigan': 24, 'short coat': 25, 'short sleeves': 26, 'shorts': 27, 'sleeveless': 28, 'solid colours': 29, 'tracksuit': 30, 'trapeze dress': 31}
{'yellow': 0, 'brown': 1, 'blue': 2, 'grey': 3, 'green': 4, 'black': 5, 'red': 6, 'white': 7, 'orange': 8, 'violet': 9}
{'acrylic': 0, 'scuba crepe': 49, 'tulle': 55, 'angora': 1, 'faux leather': 14, 'georgette': 21, 'lurex': 30, 'nice': 38, 'crepe': 9, 'satin cotton': 47, 'silky satin': 51, 'fur': 20, 'matte jersey': 33, 'plisse': 43, 'velvet': 56, 'lace': 27, 'cotton': 8, 'piquet': 42, 'plush': 45, 'bengaline': 2,

In [15]:
class TextEmbedder(nn.Module):
    def __init__(self, embedding_dim, cat_dict, col_dict, fab_dict, gpu_num):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.cat_dict = {v: k for k, v in cat_dict.items()}
        self.col_dict = {v: k for k, v in col_dict.items()}
        self.fab_dict = {v: k for k, v in fab_dict.items()}
        self.word_embedder = pipeline('feature-extraction', model='bert-base-uncased')
        self.fc = nn.Linear(768, embedding_dim)
        self.dropout = nn.Dropout(0.1)
        self.gpu_num = gpu_num

    def forward(self, category, color, fabric):
        textual_description = [self.col_dict[color.detach().cpu().numpy().tolist()[i]] + ' ' \
                + self.fab_dict[fabric.detach().cpu().numpy().tolist()[i]] + ' ' \
                + self.cat_dict[category.detach().cpu().numpy().tolist()[i]] for i in range(len(category))]


        # Use BERT to extract features
        word_embeddings = self.word_embedder(textual_description)

        # BERT gives us embeddings for [CLS] ..  [EOS], which is why we only average the embeddings in the range [1:-1] 
        # We're not fine tuning BERT and we don't want the noise coming from [CLS] or [EOS]
        word_embeddings = [torch.FloatTensor(x[0][1:-1]).mean(axis=0) for x in word_embeddings] 
        word_embeddings = torch.stack(word_embeddings).to('cuda:'+str(self.gpu_num))
        
        # Embed to our embedding space
        word_embeddings = self.dropout(self.fc(word_embeddings))

        return word_embeddings

In [19]:
text_encoder = TextEmbedder(32, cat_dict, col_dict, fab_dict, 0)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

c:\Python310\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahmed\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [25]:
"""category = torch.tensor(np.array("cloak"))
color = torch.tensor("brown")
fabric = torch.tensor("tulle")"""

category = torch.tensor(1)
color = 1
fabric = 55
text_encoding = text_encoder(category, color, fabric)

TypeError: len() of a 0-d tensor